In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import requests

import matplotlib.pyplot as plt
import mygene
import goatools

c:\Users\areeba khan\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [5]:
import requests

# URL with credentials
url = ('https://s3.amazonaws.com/data.clue.io/api/bq_creds/BQ-demo-credentials.json')

response = requests.get(url)
credentials_filepath='C:/Users/areeba khan/Documents/UdS/Master Thesis/BQ-demo-credentials.json'

with open(credentials_filepath, 'w') as f:
  f.write(response.text)

In [7]:
import cmapBQ.query as cmap_query
import cmapBQ.config as cmap_config

credentials_filepath='C:/Users/areeba khan/Documents/UdS/Master Thesis/BQ-demo-credentials.json'
# Set up credentials
cmap_config.setup_credentials(credentials_filepath)
bq_client = cmap_config.get_bq_client()

In [8]:
targets = cmap_query.list_cmap_targets(bq_client)
targets

,target,count
0,None,31262
1,NR1I3,3
2,ACVR1,3
3,AKT3,7
4,AKT1,10
...,...,...
886,WASL,1
887,EIF2S1,2
888,MTTP,1
889,HSD3B2,1


In [54]:
import json

input_file_path = "C:\\Users\\areeba khan\\Documents\\UdS\\Master Thesis\\CMAP_Output\\brca\\equal_set\\vertex\\overlap_compounds.txt"

with open(input_file_path, 'r') as file:
    data = json.load(file)

cmap_pert_list = [(entry['cmap_compunds'], entry['pert_id']) for entry in data]
cmap_pert_list = list(set(cmap_pert_list))
print(cmap_pert_list)

[('palbociclib', 'BRD-K51313569'), ('doxorubicin', 'BRD-K92093830'), ('doxorubicin', 'BRD-A76941896'), ('tamoxifen', 'BRD-K43744935'), ('tamoxifen', 'BRD-K04210847'), ('doxorubicin', 'BRD-A52530684'), ('fulvestrant', 'BRD-A83237092'), ('mitomycin-c', 'BRD-A48237631'), ('fulvestrant', 'BRD-A90490067')]


In [58]:
cmp_data = []
for cmps in cmap_pert_list:
    cmp, pert_id = cmps
    # print(cmp, pert_id)
    df = cmap_query.cmap_compounds(client = bq_client, cmap_name = cmp, pert_id = pert_id)
    filtered_df = df[["pert_id", "cmap_name", "target", "moa"]]
    # print(filtered_df)
    cmp_data.append(filtered_df)


combined_df = pd.concat(cmp_data, ignore_index=True)

final_df = combined_df.drop_duplicates(subset=["pert_id", "target", "moa"])
print("Final DataFrame after removing duplicates:")
print(final_df)
    

Final DataFrame after removing duplicates:
          pert_id    cmap_name target                                    moa
0   BRD-K51313569  palbociclib   CDK4                          CDK inhibitor
1   BRD-K51313569  palbociclib   CDK6                          CDK inhibitor
2   BRD-K92093830  doxorubicin  TOP2A                Topoisomerase inhibitor
3   BRD-A76941896  doxorubicin  TOP2A                Topoisomerase inhibitor
4   BRD-K43744935    tamoxifen   ESR1           Estrogen receptor antagonist
5   BRD-K43744935    tamoxifen   ESR2           Estrogen receptor antagonist
6   BRD-K43744935    tamoxifen   ESR1  Selective estrogen receptor modulator
7   BRD-K43744935    tamoxifen   ESR2  Selective estrogen receptor modulator
8   BRD-K04210847    tamoxifen   ESR1           Estrogen receptor antagonist
9   BRD-K04210847    tamoxifen   ESR2           Estrogen receptor antagonist
10  BRD-K04210847    tamoxifen   ESR1  Selective estrogen receptor modulator
11  BRD-K04210847    tamoxifen   

In [59]:
mg = mygene.MyGeneInfo()

def get_go_annotations(gene_symbols):
    go_annotations = {}
    for gene in gene_symbols:
        gene_info = mg.query(gene, scopes='symbol', fields='go', species='human')
        if 'hits' in gene_info and len(gene_info['hits']) > 0:
            go_data = gene_info['hits'][0].get('go', {})
            go_annotations[gene] = go_data
        else:
            go_annotations[gene] = None
    return go_annotations


unique_targets = final_df['target'].unique()

go_annotations = get_go_annotations(unique_targets)
# print(go_annotations)
def fetch_go_terms(target):
    go_data = go_annotations.get(target, {})
    
    # Initialize BP and MF GO term lists
    go_bp_terms = []
    go_mf_terms = []
    
    if not go_data:
        return None, None
    
    # Extract BP and MF terms
    for category in ['BP', 'MF']:
        category_data = go_data.get(category, [])
        
        # Handle list of dictionaries
        if isinstance(category_data, list):
            terms = [term['term'] for term in category_data if 'term' in term]
            
            # Separate BP and MF terms based on category
            if category == 'BP':
                go_bp_terms.extend(terms)
            elif category == 'MF':
                go_mf_terms.extend(terms)
        
        # Handle single dictionary (in case of a single GO term)
        elif isinstance(category_data, dict):
            if 'term' in category_data:
                term = category_data['term']
                if category == 'BP':
                    go_bp_terms.append(term)
                elif category == 'MF':
                    go_mf_terms.append(term)
        
        else:
            print(f"Warning: Unexpected format for {category}: {category_data}")
    
    # Return BP and MF GO terms as semi-colon separated strings
    return "; ".join(go_bp_terms) if go_bp_terms else None, "; ".join(go_mf_terms) if go_mf_terms else None

final_df[['GO_BP', 'GO_MF']] = final_df['target'].apply(lambda x: pd.Series(fetch_go_terms(x)))

print(final_df)


          pert_id    cmap_name target                                    moa  \
0   BRD-K51313569  palbociclib   CDK4                          CDK inhibitor   
1   BRD-K51313569  palbociclib   CDK6                          CDK inhibitor   
2   BRD-K92093830  doxorubicin  TOP2A                Topoisomerase inhibitor   
3   BRD-A76941896  doxorubicin  TOP2A                Topoisomerase inhibitor   
4   BRD-K43744935    tamoxifen   ESR1           Estrogen receptor antagonist   
5   BRD-K43744935    tamoxifen   ESR2           Estrogen receptor antagonist   
6   BRD-K43744935    tamoxifen   ESR1  Selective estrogen receptor modulator   
7   BRD-K43744935    tamoxifen   ESR2  Selective estrogen receptor modulator   
8   BRD-K04210847    tamoxifen   ESR1           Estrogen receptor antagonist   
9   BRD-K04210847    tamoxifen   ESR2           Estrogen receptor antagonist   
10  BRD-K04210847    tamoxifen   ESR1  Selective estrogen receptor modulator   
11  BRD-K04210847    tamoxifen   ESR2  S

In [60]:
final_df.to_csv("C:\\Users\\areeba khan\\Documents\\UdS\\Master Thesis\\CMAP_Output\\brca\\equal_set\\vertex\\overlap_compounds_go_term.csv")

In [31]:
cmap_query.cmap_compounds(client=bq_client, cmap_name="zonisamide", pert_id="BRD-K48300629")

,pert_id,cmap_name,target,moa,canonical_smiles,inchi_key,compound_aliases
0,BRD-K48300629,zonisamide,SCN11A,Sodium channel blocker,NS(=O)(=O)Cc1noc2ccccc12,UBQNRHZMVUUOMG-UHFFFAOYSA-N,None
1,BRD-K48300629,zonisamide,SCN1A,Sodium channel blocker,NS(=O)(=O)Cc1noc2ccccc12,UBQNRHZMVUUOMG-UHFFFAOYSA-N,None
2,BRD-K48300629,zonisamide,SCN2A,Sodium channel blocker,NS(=O)(=O)Cc1noc2ccccc12,UBQNRHZMVUUOMG-UHFFFAOYSA-N,None
3,BRD-K48300629,zonisamide,SCN3A,Sodium channel blocker,NS(=O)(=O)Cc1noc2ccccc12,UBQNRHZMVUUOMG-UHFFFAOYSA-N,None
4,BRD-K48300629,zonisamide,SCN4A,Sodium channel blocker,NS(=O)(=O)Cc1noc2ccccc12,UBQNRHZMVUUOMG-UHFFFAOYSA-N,None
5,BRD-K48300629,zonisamide,SCN5A,Sodium channel blocker,NS(=O)(=O)Cc1noc2ccccc12,UBQNRHZMVUUOMG-UHFFFAOYSA-N,None
6,BRD-K48300629,zonisamide,SCN9A,Sodium channel blocker,NS(=O)(=O)Cc1noc2ccccc12,UBQNRHZMVUUOMG-UHFFFAOYSA-N,None
7,BRD-K48300629,zonisamide,CA1,Sodium channel blocker,NS(=O)(=O)Cc1noc2ccccc12,UBQNRHZMVUUOMG-UHFFFAOYSA-N,None
8,BRD-K48300629,zonisamide,CA7,Sodium channel blocker,NS(=O)(=O)Cc1noc2ccccc12,UBQNRHZMVUUOMG-UHFFFAOYSA-N,None
9,BRD-K48300629,zonisamide,CA12,Sodium channel blocker,NS(=O)(=O)Cc1noc2ccccc12,UBQNRHZMVUUOMG-UHFFFAOYSA-N,None


In [24]:
df = cmap_query.cmap_genetic_perts(client=bq_client)
df.to_csv("C:/Users/areeba khan/Documents/UdS/Master Thesis/gene_info.csv")

In [27]:
cmap_query.cmap_genetic_perts(client=bq_client, cmap_name="ADRB2")

,pert_id,cmap_name,pert_type,gene_id,gene_title,ensembl_id,gene_type,feature_space
0,BRDN0000404622,ADRB2,trt_oe,154,adrenoceptor beta 2,ENSG00000169252,protein-coding,landmark
1,TRCN0000008084,ADRB2,trt_sh,154,adrenoceptor beta 2,ENSG00000169252,protein-coding,landmark
2,TRCN0000008085,ADRB2,trt_sh,154,adrenoceptor beta 2,ENSG00000169252,protein-coding,landmark
3,TRCN0000008086,ADRB2,trt_sh,154,adrenoceptor beta 2,ENSG00000169252,protein-coding,landmark
4,CGS001-154,ADRB2,trt_sh.cgs,154,adrenoceptor beta 2,ENSG00000169252,protein-coding,landmark
5,BRDN0001479155,ADRB2,trt_xpr,154,adrenoceptor beta 2,ENSG00000169252,protein-coding,landmark
6,BRDN0001486168,ADRB2,trt_xpr,154,adrenoceptor beta 2,ENSG00000169252,protein-coding,landmark
7,BRDN0001486383,ADRB2,trt_xpr,154,adrenoceptor beta 2,ENSG00000169252,protein-coding,landmark
